In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords, wordnet
from collections import Counter
import re
from part_of_speech import get_part_of_speech






In [4]:
df = pd.read_csv('../okcupiddata/profiles.csv')
df.columns

Index(['age', 'body_type', 'diet', 'drinks', 'drugs', 'education', 'essay0',
       'essay1', 'essay2', 'essay3', 'essay4', 'essay5', 'essay6', 'essay7',
       'essay8', 'essay9', 'ethnicity', 'height', 'income', 'job',
       'last_online', 'location', 'offspring', 'orientation', 'pets',
       'religion', 'sex', 'sign', 'smokes', 'speaks', 'status'],
      dtype='object')

In [6]:
df['essaycorpus'] = df.essay0 + df.essay1 + df.essay2 + df.essay3 + df.essay4 + df.essay5 + df.essay6 + df.essay7 + df.essay8 + df.essay9

59946


In [32]:
dfnew = df
dfnew['essay2'] = dfnew['essay2'].str.replace(r'<[^<]+?>', '')
dfnew['essay2'] = dfnew['essay2'].str.replace(r'[^\w\s]+', ' ')
dfnew['essay2'] = dfnew['essay2'].replace(r'\r+|\n+|\t+',' ', regex=True)

print(dfnew.head(10))

   age       body_type               diet      drinks      drugs  \
0   22  a little extra  strictly anything    socially      never   
1   35         average       mostly other       often  sometimes   
2   38            thin           anything    socially        NaN   
3   23            thin         vegetarian    socially        NaN   
4   29        athletic                NaN    socially      never   
5   29         average    mostly anything    socially        NaN   
6   32             fit  strictly anything    socially      never   
7   31         average    mostly anything    socially      never   
8   24             NaN  strictly anything    socially        NaN   
9   37        athletic    mostly anything  not at all      never   

                           education  \
0      working on college/university   
1              working on space camp   
2     graduated from masters program   
3      working on college/university   
4  graduated from college/university   
5  graduate

In [ ]:
df['text'] = df['text'].str.replace(r'<[^<]+?>', '')
df['text'] = df['text'].str.replace(r'[^\w\s]+', '')

